In [1]:
from gurobipy import*
import pandas as pd
import re


#建構Model https://www.gurobi.com/documentation/9.5/refman/py_model2.html 
m = Model()  # Model ( name="", env=defaultEnv )


#定義車班類別
class train:
    def __init__(self, SS, ES, ST,ET):
        self.SS = SS
        self.ES = ES
        self.ST = ST
        self.ET = ET
    def get_value(self):
         print("； 出發站 :", self.SS, "； 到達站 :", self.ES, "； 出發時間 :", self.ST, "； 到達時間 :", self.ET)


# 讀取車班資料
df = pd.read_csv('TRA1.csv', sep=",")
df = df[df['INDEX']<=50]

# 將每個車班資料的類別逐一定義並放進 List 中。
T = []
for data in df.iterrows():
    data = data[1]
    T.append(train(data['ID_FROM'],data['ID_TO'],data['TIME_START'],data['TIME_END']))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-09


In [4]:
T[0].get_value()

； 出發站 : 1 ； 到達站 : 2 ； 出發時間 : 1178 ； 到達時間 : 1427


In [5]:
T_test = [T[i] for i in range(10)]

In [9]:
print(T_test[2].get_value())

； 出發站 : 1 ； 到達站 : 4 ； 出發時間 : 854 ； 到達時間 : 910
None


In [10]:
print(T_test[3].get_value())

； 出發站 : 4 ； 到達站 : 1 ； 出發時間 : 1032 ； 到達時間 : 1124
None


In [11]:
for i in range(len(T_test)):
    print(T_test[i].get_value())

； 出發站 : 1 ； 到達站 : 2 ； 出發時間 : 1178 ； 到達時間 : 1427
None
； 出發站 : 2 ； 到達站 : 1 ； 出發時間 : 685 ； 到達時間 : 928
None
； 出發站 : 1 ； 到達站 : 4 ； 出發時間 : 854 ； 到達時間 : 910
None
； 出發站 : 4 ； 到達站 : 1 ； 出發時間 : 1032 ； 到達時間 : 1124
None
； 出發站 : 1 ； 到達站 : 3 ； 出發時間 : 336 ； 到達時間 : 512
None
； 出發站 : 3 ； 到達站 : 1 ； 出發時間 : 647 ； 到達時間 : 842
None
； 出發站 : 1 ； 到達站 : 5 ； 出發時間 : 827 ； 到達時間 : 994
None
； 出發站 : 5 ； 到達站 : 1 ； 出發時間 : 1025 ； 到達時間 : 1208
None
； 出發站 : 1 ； 到達站 : 4 ； 出發時間 : 408 ； 到達時間 : 492
None
； 出發站 : 4 ； 到達站 : 1 ； 出發時間 : 538 ； 到達時間 : 639
None


In [15]:
# 前10台車裡面，除了將第2, 3台合併外，其他一樣
import numpy as np
T_ans = []
num = np.array([[[0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]],[[0,0,0,0,0,0,0,0,0,1],[0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]]])
num.shape
        

(2, 3, 10)

In [ ]:
for i in range(num.shape[0]):
    start = 0
    end = 0
    for j in range(num.shape[1]):
        for k in range(num.shape[2]):
            if (num[i,j,k]==1):
                end += 1



In [17]:
import pandas as pd

grades = {
    "name": ["Mike", "Sherry", "Cindy", "John"],
    "math": [80, 75, 93, 86],
    "chinese": [63, 90, 85, 70]
}
 
df = pd.DataFrame(grades)
df['what'] = 1
df


,name,math,chinese,what
0,Mike,80,63,1
1,Sherry,75,90,1
2,Cindy,93,85,1
3,John,86,70,1
